# Exports workflow

## Libraries

Starting loading with some libraries

In [58]:
import pandas as pd
import numpy as np
import os
import datetime as dt

Now we check the files in temporal folder

In [3]:
path = os.path.join(os.path.dirname(os.getcwd()), "temp")
path_con = [e for e in os.listdir(path) if e.startswith('con_data')][0]
path_chp = [e for e in os.listdir(path) if e.startswith('cap_data')][0]
print([path_con, path_chp])

['con_data_2022_03_16__21_06_03.xls', 'cap_data_2022_03_16__21_06_42.xls']


## cleaning new data

We clean the data

In [22]:
data_con_raw = pd.read_excel(os.path.join(path, path_con), skiprows = 1)
data_con_raw = data_con_raw.rename(columns={data_con_raw.columns[0] : "country"})
cols_expected = [str(e) for e in list(range(1990, 2030)) + ["country"]]
cols2drop = [e for e in data_con_raw.columns if e not in cols_expected]
cols2drop = [e for e in cols2drop if not e.startswith('202')]
data_con_raw = data_con_raw.drop(columns=cols2drop, axis = 0)
# data_con_raw = data_con_raw.set_index("country")

WARNING *** file size (192128) not 512 + multiple of sector size (512)


Now we convert the data to long

In [34]:
data_con_long = pd.melt(data_con_raw, id_vars="country", var_name="year")
data_con_long["value"] = data_con_long["value"].replace({"NaN": np.nan})
data_con_long = data_con_long.query("value.notna()")
data_con_long

,country,year,value
2,Afganistán,1998,0.000000e+00
3,Albania,1998,0.000000e+00
4,Alemania,1998,0.000000e+00
5,Andorra,1998,0.000000e+00
9,Antigua y Barbuda,1998,0.000000e+00
...,...,...,...
5884,Ucrania,2022*,1.032729e+03
5886,Uruguay,2022*,4.318386e+02
5889,Venezuela,2022*,1.270334e+03
5890,Vietnam,2022*,1.128535e+03


Fixing some countries entries

In [37]:
data_con_long["country"] = data_con_long["country"].replace({"Reino Unido-No UE": "Reino Unido",
 "Reino Unido-UE" : "Reino Unido"
})
data_con_long = data_con_long.query("country != 'Grand Total'")

Now, we add the current date

In [44]:
data_con_long["time"] = dt.datetime.now()
data_con_long

C:\Users\Marlon.molina\AppData\Local\Temp\ipykernel_27560\2596054046.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_con_long["time"] = dt.datetime.now()


,country,year,value,time
2,Afganistán,1998,0.00000,2022-04-12 20:33:16.082471
3,Albania,1998,0.00000,2022-04-12 20:33:16.082471
4,Alemania,1998,0.00000,2022-04-12 20:33:16.082471
5,Andorra,1998,0.00000,2022-04-12 20:33:16.082471
9,Antigua y Barbuda,1998,0.00000,2022-04-12 20:33:16.082471
...,...,...,...,...
5883,Turquía,2022*,1891.72654,2022-04-12 20:33:16.082471
5884,Ucrania,2022*,1032.72948,2022-04-12 20:33:16.082471
5886,Uruguay,2022*,431.83857,2022-04-12 20:33:16.082471
5889,Venezuela,2022*,1270.33431,2022-04-12 20:33:16.082471


Now we aggregate the data to obtain the new values to compare

In [123]:
data_con_new = data_con_long.groupby(["year"]).agg({"value" : np.sum})
data_con_new = data_con_new.rename(columns={"value":"new"})
data_con_new.index = data_con_new.index.astype("str").str.strip()

## Loading historical data

The next step is to check if there is any historical (must be in this pipeline) and append the new data

In [87]:
path_con_hist = os.path.join(os.path.dirname(os.getcwd()), "data", "historical_country_data_procomer.csv")
data_con_hist = pd.read_csv(path_con_hist, delimiter='|')
data_con_hist2 = data_con_hist.append(data_con_long)
data_con_hist2.to_csv(path_con_hist, sep='|', index=False)

C:\Users\Marlon.molina\AppData\Local\Temp\ipykernel_27560\1786779730.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_con_hist2 = data_con_hist.append(data_con_long)


Now, its time to filter with the last timestamp and group the data by year

In [124]:
last_time = data_con_hist["time"].min()
data_con_old = data_con_hist.query("time == @last_time")
data_con_old = data_con_old.groupby(["year"]).agg({"value" : np.sum})
data_con_old = data_con_old.rename(columns={"value":"old"})
data_con_old.index = data_con_old.index.astype("str").str.strip()

## Compare old and new data frame

Now we assign a tol value. Then we used to create a check column

In [147]:
tol = 0.001
con_join = pd.merge(data_con_old, data_con_new, how="outer", left_index=True, right_index=True)
con_join[["old", "new"]] = con_join[["old", "new"]].replace({np.nan : 0})
con_join["check"] = abs(con_join["old"] - con_join["new"])
con_join["check"] = con_join["check"] <= tol
con_status = con_join
con_check = con_join.query("~ check")

Now we save both, the status and check data frames

In [ ]:
path_con_status = os.path.join(os.path.dirname(os.getcwd()), "data", "status_procomer.csv")
path_con_check = os.path.join(os.path.dirname(os.getcwd()), "data", "check_procomer.csv")
con_status.to_csv(path_con_status, sep='|', index=False)
con_check.to_csv(path_con_check, sep='|', index=False)